In [0]:
import pandas as pd 
import numpy as np
from sklearn import preprocessing

In [0]:
olymipics = pd.read_csv('/content/drive/My Drive/athlete_events.csv')
print(olymipics.shape)

In [0]:
print(olymipics.isnull().sum())

In [0]:
olymipics['Medal'].fillna('DNW',inplace = True)

In [0]:
noc_country = pd.read_csv('/content/drive/My Drive/noc_regions.csv')
noc_country.drop('notes',axis=1,inplace=True)
noc_country.rename(columns={'region':'Country'},inplace=True)

In [0]:
olymipics_merge = olymipics.merge(noc_country,left_on='NOC',right_on="NOC",how="left")

In [0]:
print(olymipics_merge.loc[olymipics_merge['Country'].isnull(),['NOC','Team']].drop_duplicates())

In [0]:
olymipics_merge.loc[olymipics_merge['Country'].isnull(),['Country']]=olymipics_merge['Team']
olymipics_merge['Country']= np.where(olymipics_merge['NOC']=='SGP','Singapore',olymipics_merge['Country'])
olymipics_merge['Country']=np.where(olymipics_merge['NOC']=='ROT','Refugee Olympic Athletes',olymipics_merge['Country'])
olymipics_merge['Country']=np.where(olymipics_merge['NOC']=='UNK','Unknown',olymipics_merge['Country'])
olymipics_merge['Country']=np.where(olymipics_merge['NOC']=='TUV','Tuvalu',olymipics_merge['Country'])

In [0]:
for i in olymipics_merge.columns:
  print(i)

In [0]:
olymipics_merge.drop('Team',axis=1,inplace=True)
olymipics_merge.rename(columns={'Country','Team'},inplace=True)

In [0]:
print(olymipics_merge.isnull().sum())

In [0]:
#taking data of summer olympics from 1961 onwards
olymipics_subset=olymipics_merge.loc[(olymipics_merge['Year']>1960) & (olymipics_merge['Season']=='Summer'),:]
print(olymipics_subset.shape)

In [0]:
#extracting unique element in a variable list_unique
list_unique=olymipics_subset.Event.unique()
print(list_unique.shape)
print(list_unique)

In [0]:
#mean of specific event will be taken and null values will be replaced by that 
tempagemean=[]
tempweightmean=[]
tempheightmean=[]

In [0]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [0]:
for i in list_unique:
  temporary = olymipics_subset.loc[(olymipics_subset['Event']==i)&(olymipics_subset['Age']!='NaN'),:]
  tempagemean.append(temporary.Age.mean(axis =0,skipna=True))
  tempweightmean.append(temporary.Weight.mean(axis =0,skipna=True))
  tempheightmean.append(temporary.Height.mean(axis =0,skipna=True))

In [0]:
print(len(tempagemean))
print(len(tempheightmean))
print(len(tempweightmean))

In [0]:
pd.set_option('mode.chained_assignment', None)


In [0]:
for i in range(len(list_unique)):
  df1=olymipics_subset[olymipics_subset['Event'].str.contains(list_unique[i])]
  df1.Age.fillna(tempagemean[i],inplace=True)
  df1.Weight.fillna(tempagemean[i],inplace=True)
  df1.Height.fillna(tempagemean[i],inplace=True)

In [0]:
print(df1.shape)

In [0]:
olymipics_subset.drop(olymipics_subset[olymipics_subset['Event'].str.contains(list_unique[i])].index , inplace=True)
olymipics_subset=pd.concat([olymipics_subset,df1],axis=0) 

In [0]:
print(olymipics_subset.isnull().sum())

In [0]:
olymipics_subset['Sex'].replace('M','1')